<a href="https://colab.research.google.com/github/KaifAhmad1/Agri-Llama/blob/main/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing Necessary Dependencies:**

In [19]:
!pip install -qU bitsandbytes
!pip install -qU trl
!pip install -qU transformers
!pip install -qU peft
!pip install -qU optimum
!pip install -qU datasets
!pip install -qU accelerate
!pip install -qU nltk
!pip install -qU rouge_score

**Necessary Imports:**

In [20]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    BitsAndBytesConfig,
    pipeline,
    logging
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from datasets import Dataset
from huggingface_hub import notebook_login
from google.colab import drive
import plotly.express as px
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

**Set Up Environment:**

In [21]:
notebook_login()
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Load Data:**

In [4]:
# Load data
file_path = '/content/drive/MyDrive/Network-QA-Dataset.csv'
data = pd.read_csv(file_path)
data

,Questions,Answers,Context Info,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106,Unnamed: 107,Unnamed: 108,Unnamed: 109,Unnamed: 110
0,What is the scope of the technical specificati...,The scope of the technical specification is de...,"The technical specification, titled ""3GPP TS 2...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Where can specifications and reports for the i...,Specifications and reports for the implementat...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,What are the different restoration indicators ...,The document discusses various restoration ind...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,What procedures are outlined for the restorati...,Procedures for the restoration of data in the ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,In which section can information about the res...,Information about the restoration of data in ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1266,"In the context of CAPIF deployment models, wha...","""NEF implements the CAPIF architecture"" means...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1267,"Explain the concept of ""Distributed deployment...","The ""Distributed deployment of the NEF complia...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1268,"According to Annex D, what is the document's a...",Annex D provides a table (Table D-1) that illu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1269,What kind of information does Annex E (Configu...,Annex E specifies configuration data for CAPIF...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
network_data = data[['Questions', 'Answers', 'Context Info']]
network_data

,Questions,Answers,Context Info
0,What is the scope of the technical specificati...,The scope of the technical specification is de...,"The technical specification, titled ""3GPP TS 2..."
1,Where can specifications and reports for the i...,Specifications and reports for the implementat...,NaN
2,What are the different restoration indicators ...,The document discusses various restoration ind...,NaN
3,What procedures are outlined for the restorati...,Procedures for the restoration of data in the ...,NaN
4,In which section can information about the res...,Information about the restoration of data in ...,NaN
...,...,...,...
1266,"In the context of CAPIF deployment models, wha...","""NEF implements the CAPIF architecture"" means...",NaN
1267,"Explain the concept of ""Distributed deployment...","The ""Distributed deployment of the NEF complia...",NaN
1268,"According to Annex D, what is the document's a...",Annex D provides a table (Table D-1) that illu...,NaN
1269,What kind of information does Annex E (Configu...,Annex E specifies configuration data for CAPIF...,NaN


In [6]:
def process_data_sample(example):
    # Extract relevant information from the example
    question = example['Questions']
    answer = example['Answers']
    context_info = example['Context Info']

    # Convert potential NaN values to empty string
    question = str(question)
    answer = str(answer)
    context_info = str(context_info) if pd.notna(context_info) else ""

    # Prepare the processed example for a Question Answering System
    processed_example = (
        "You are a Question Answering System designed to assist users with queries. "
        "Your capabilities include providing technical details, offering implementation guidance, "
        "and staying updated on telecommunications standards.\n\n"
        f"User Query:\n{question}\n\n"
        f"Answer:\n{answer}\n\n"
        f"Context Information:\n{context_info}"
    )
    return processed_example

In [7]:
# Create 'text' column in 'network_data' by applying 'process_data_sample' to each row's 'Questions', 'Answers', and 'Context Info'
network_data['text'] = network_data[['Questions', 'Answers', 'Context Info']].apply(lambda x: process_data_sample(x), axis=1)

<ipython-input-7-7a9adacdc3c7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  network_data['text'] = network_data[['Questions', 'Answers', 'Context Info']].apply(lambda x: process_data_sample(x), axis=1)


In [22]:
for i in range(5):
  print(network_data['text'].iloc[i])

You are a Question Answering System designed to assist users with queries. Your capabilities include providing technical details, offering implementation guidance, and staying updated on telecommunications standards.

User Query:
What is the scope of the technical specification mentioned in the document?

Answer:
The scope of the technical specification is defined in Section 1, and it covers restoration procedures within the 3rd Generation Partnership Project (3GPP) for Core Network and Terminals, Release 18.

Context Information:
The technical specification, titled "3GPP TS 23.007 V18.2.0 (2023-06)," is a document developed within the 3rd Generation Partnership Project (3GPP). It falls under the Technical Specification Group Core Network and Terminals and specifically focuses on "Restoration procedures" as part of Release 18.The document outlines various aspects related to restoration procedures within the 3GPP framework. It covers a wide range of topics, including restoration indicat

In [23]:
# Split data
train_data, test_data = train_test_split(network_data, test_size=0.2, random_state=42)

In [24]:
# Create datasets for training and evaluation using the datasets library
network_train_data = Dataset.from_pandas(train_data)
network_test_data = Dataset.from_pandas(test_data)

In [25]:
network_train_data

Dataset({
    features: ['Questions', 'Answers', 'Context Info', 'text', '__index_level_0__'],
    num_rows: 1016
})

In [26]:
network_test_data

Dataset({
    features: ['Questions', 'Answers', 'Context Info', 'text', '__index_level_0__'],
    num_rows: 255
})

In [12]:
model_name = 'mistralai/Mistral-7B-v0.1'

In [27]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [28]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    low_cpu_mem_usage=True
)

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
# Tokenization and Padding
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
)
tokenizer.pad_token = tokenizer.eos_token

**LoRA and SFT**

In [ ]:
model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
# LoRA Config
peft_config = LoraConfig(
    r=64,
    lora_alpha=16,
    lora_dropout=0.1,
    bias='none',
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"]
)

In [ ]:
# Training Arguments
training_arguments = TrainingArguments(
    output_dir='Mistral-Network-QnA-System',
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim='paged_adamw_32bit',
    learning_rate=2e-4,
    lr_scheduler_type='cosine',
    save_strategy='epoch',
    logging_steps=10,
    save_steps=10,
    num_train_epochs=1,
    max_steps=200,
    fp16=True,
    warmup_ratio=0.05,
    push_to_hub=False,
)

In [ ]:
# SFT Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=Dataset.from_pandas(network_train_data[['text']]),
    peft_config=peft_config,
    dataset_text_field='text',
    args=training_arguments,
    tokenizer=tokenizer,
    packing=False,
    max_seq_length=264
)

In [ ]:
# Train the model
trainer.train()

In [ ]:
import plotly.express as px

# Extract training loss values
train_losses = [entry['loss'] for entry in trainer.state.log_history if 'loss' in entry]

# Create a plot
fig = px.line(x=range(1, len(train_losses) + 1), y=train_losses, title='Training Loss Over Steps',
              labels={'x': 'Steps', 'y': 'Training Loss'})

# Show the plot
fig.show()